In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import tensorflow as tf
from skimage import exposure
#from tensorflow.contrib.layers import flatten
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Flatten

from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
import cv2
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def rotateImage(img, angle):
    (rows, cols, ch) = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2), angle, 1)
    return cv2.warpAffine(img, M, (cols,rows))
    
    
def loadBlurImg(path, imgSize):
    img = cv2.imread(path)
    angle = np.random.randint(0, 360)
    img = rotateImage(img, angle)
    img = cv2.blur(img,(5,5))
    img = cv2.resize(img, imgSize)
    return img

def loadImgClass(classPath, classLable, classSize, imgSize):
    x = []
    y = []
    
    for path in classPath:
        img = loadBlurImg(path, imgSize)        
        x.append(img)
        y.append(classLable)
        
    while len(x) < classSize:
        randIdx = np.random.randint(0, len(classPath))
        img = loadBlurImg(classPath[randIdx], imgSize)
        x.append(img)
        y.append(classLable)
        
    return x, y

def loadData(img_size, classSize):
    keloids = glob.glob('/content/drive/MyDrive/Hack-the-globe/Keloiods-yes/**/*.jpeg', recursive=True) 
    notKeloids = glob.glob('/content/drive/MyDrive/Hack-the-globe/Keloids-not/**/*.jpeg', recursive=True)
    
    
    imgSize = (img_size, img_size)
    xKeloid, yKeloid = loadImgClass(keloids, 0, classSize, imgSize)
    xNotKeloid, yNotKeloid = loadImgClass(notKeloids, 1, classSize, imgSize)
    print("There are", len(xKeloid), "Keloids images")
    print("There are", len(xNotKeloid), "not Keloids images")
    
    X = np.array(xKeloid + xNotKeloid)
    y = np.array(yKeloid + yNotKeloid)
    
    return X, y

In [4]:
def toGray(images):
    # rgb2gray converts RGB values to grayscale values by forming a weighted sum of the R, G, and B components:
    # 0.2989 * R + 0.5870 * G + 0.1140 * B 
    # source: https://www.mathworks.com/help/matlab/ref/rgb2gray.html
    
    images = 0.2989*images[:,:,:,0] + 0.5870*images[:,:,:,1] + 0.1140*images[:,:,:,2]
    return images

def normalizeImages(images):
    # use Histogram equalization to get a better range
    # source http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.equalize_hist
    images = (images / 255.).astype(np.float32)
    
    for i in range(images.shape[0]):
        images[i] = exposure.equalize_hist(images[i])
    
    images = images.reshape(images.shape + (1,)) 
    return images

def preprocessData(images):
    grayImages = toGray(images)
    return normalizeImages(grayImages)

def normalizeImages2(images):
    for i in range(images.shape[0]):
        cv2.normalize(images[i],images[i], alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    # if convert to gray scale use this after
    print("images has shape before", images.shape)
    #images = images.reshape(images.shape + (1,)) 
    #print("images has shape after", images.shape)
    return images

In [5]:
size = 128 #image size
classSize = 15000 

scaled_X, y = loadData(size, classSize)
scaled_X = preprocessData(scaled_X)

n_classes = len(np.unique(y))

from keras.utils.np_utils import to_categorical
y = to_categorical(y)

There are 15000 Keloids images
There are 15000 not Keloids images


In [6]:
rand_state = np.random.randint(0, 100)

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

print("Number of classes =", n_classes)
print("train shape X", X_train.shape)
print("train shape y", y_train.shape)

Number of classes = 2
train shape X (24000, 128, 128, 1)
train shape y (24000, 2)


In [7]:
def karasModel(inputShape):
    model = Sequential()
    model.add(Conv2D(16, 8, 8, subsample=(4, 4),border_mode='valid', input_shape=inputShape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    model.add(Conv2D(32, 5, 5))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    model.add(Conv2D(32, 3, 3))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(240))

    model.add(Activation('relu'))
    model.add(Dense(120))

    #model.add(Activation('relu'))
    model.add(Dense(2))

    model.add(Activation('softmax'))
    return model 

def karasModel2(inputShape):
    model = Sequential()
    #model.add(Conv2D(16, 8, 8, subsample=(4, 4),border_mode='valid', input_shape=inputShape))
    model.add(Conv2D(16, 8, 8, padding='valid', input_shape=inputShape))
    model.add(ELU())
    model.add(Conv2D(32, 5, 5, padding="same"))
    model.add(ELU())
    model.add(Conv2D(64, 5, 5, padding="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(2))
    model.add(Activation('softmax'))
    return model

In [8]:
from keras.layers.convolutional import Conv2D

inputShape = (size, size, 1)
model = karasModel2(inputShape)


model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train, epochs=50, validation_split=0.1) 


metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

Epoch 1/50
675/675 [==============================] - 8s 10ms/step - loss: 0.6228 - accuracy: 0.6505 - val_loss: 0.5629 - val_accuracy: 0.7104
Epoch 2/50
675/675 [==============================] - 9s 13ms/step - loss: 0.5492 - accuracy: 0.7206 - val_loss: 0.5531 - val_accuracy: 0.7179
Epoch 3/50
675/675 [==============================] - 7s 10ms/step - loss: 0.5031 - accuracy: 0.7543 - val_loss: 0.5289 - val_accuracy: 0.7321
Epoch 4/50
675/675 [==============================] - 7s 10ms/step - loss: 0.4622 - accuracy: 0.7773 - val_loss: 0.4904 - val_accuracy: 0.7487
Epoch 5/50
675/675 [==============================] - 7s 10ms/step - loss: 0.4180 - accuracy: 0.7989 - val_loss: 0.5750 - val_accuracy: 0.7433
Epoch 6/50
675/675 [==============================] - 7s 10ms/step - loss: 0.3787 - accuracy: 0.8240 - val_loss: 0.4089 - val_accuracy: 0.8046
Epoch 7/50
675/675 [==============================] - 7s 10ms/step - loss: 0.3489 - accuracy: 0.8400 - val_loss: 0.3899 - val_accuracy: 0.8167